# CNE-join : 조인 가능한 테이블을 찾아주는 코드

In [1]:
##개체명 인식기는 https://towardsdatascience.com/named-entity-recognition-with-bert-in-pytorch-a454405e0b6a 를 참고하여 만듦
import pandas as pd
df = pd.read_csv('ner.csv')
# Split labels based on whitespace and turn them into a list
labels = [i.split() for i in df['labels'].values.tolist()]

# Check how many labels are there in the dataset
unique_labels = set()

for lb in labels:
  [unique_labels.add(i) for i in lb if i not in unique_labels]
# Map each label into its id representation and vice versa
labels_to_ids = {k: v for v, k in enumerate(sorted(unique_labels))}
ids_to_labels = {v: k for v, k in enumerate(sorted(unique_labels))}

#필요한 모듈 임포트
from pandas.api.types import is_numeric_dtype
import torch
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader

from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

from transformers import BertForTokenClassification

class BertModel(torch.nn.Module):

    def __init__(self):

        super(BertModel, self).__init__()

        self.bert = BertForTokenClassification.from_pretrained('bert-base-cased', num_labels=len(unique_labels))

    def forward(self, input_id, mask, label):

        output = self.bert(input_ids=input_id, attention_mask=mask, labels=label, return_dict=False)

        return output
    
def align_word_ids(texts):
  
    tokenized_inputs = tokenizer(texts, padding='max_length', max_length=512, truncation=True)

    word_ids = tokenized_inputs.word_ids()

    previous_word_idx = None
    label_ids = []

    for word_idx in word_ids:

        if word_idx is None:
            label_ids.append(-100)

        elif word_idx != previous_word_idx:
            try:
                label_ids.append(1)
            except:
                label_ids.append(-100)
        else:
            try:
                label_ids.append(1 if label_all_tokens else -100)
            except:
                label_ids.append(-100)
        previous_word_idx = word_idx

    return label_ids


In [7]:
# Fasttext 모델 임포트
from gensim.models import fasttext

ft_model = fasttext.load_facebook_model("cc.en.300.bin")

# 개체명인식 모델 임포트
model = torch.load("model_alldata.pth")

In [10]:
def evaluate_list(model, arr):
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    
    #geo for geographical entity / org for organization entity / per for person entity / gpe for geopolitical entity / tim for time indicator entity / art for artifact entity / eve for event entity / nat for natural phenomenon entity / O is assigned if a word doesn’t belong to any entity.
    ner_dict = {'B-geo':0, 'B-gpe':0, 'B-per':0, 'B-tim':0, 'B-eve':0, 'B-art':0, 'B-nat':0, 'B-org':0}
    if use_cuda:
        model = model.cuda()
    for sentence in arr:
        text = tokenizer(sentence, padding='max_length', max_length = 512, truncation=True, return_tensors="pt")

        mask = text['attention_mask'].to(device)
        input_id = text['input_ids'].to(device)
        label_ids = torch.Tensor(align_word_ids(sentence)).unsqueeze(0).to(device)

        logits = model(input_id, mask, None)
        logits_clean = logits[0][label_ids != -100]

        predictions = logits_clean.argmax(dim=1).tolist()
        prediction_label = [ids_to_labels[i] for i in predictions]
        for label in prediction_label:
            if label in ner_dict :
                ner_dict[label] +=1 
    return [k for k,v in ner_dict.items() if max(ner_dict.values()) == v]
    
def containment(arr1,arr2):
    #모든 값 소문자로 변경
    larr1 = set([str(i).lower() for i in arr1])
    larr2 = set([str(i).lower() for i in arr2])
    
    threshold = 0.6
    denominator = min(len(larr1), len(larr2))
    numerator = len(larr1.intersection(larr2))
    
    if (numerator / denominator) > threshold:
        return True
    else:
        return False
    
#한 테이블의 모든 컬럼에 대해 각 컬럼의 unique한 값으로 column nmae + is + column value의 문장을 만들어 그 결과를 리스트에 담아 return 하는 함수
#이를 활용해 각 컬럼의 NE를 결정한다
def make_sent(tab1, col1):
    sent1 = []
    for word in tab1[col1].unique():
        sent1.append(col1 + " is " + str(word))

    return sent1

# 두 컬럼에 대해 column_name embedding 값 반환
def column_name_emb(col1, col2):
    return ft_model.wv.similarity(col1,col2)

def joinable_table(tab1, tab2):
    #threshold 값 정의
    col_th = 0.3
    
    #각 테이블에서 수치형 컬럼 전부 드랍
    drop_col1 = []
    drop_col2 = []
    for i in range(len(tab1.columns)):
        col_name1 = tab1.columns[i]
        if is_numeric_dtype(tab1[col_name1]):
            drop_col1.append(col_name1)
    for drop in drop_col1:        
        tab1.drop(columns=[drop],inplace=True)
        
    for i in range(len(tab2.columns)):
        col_name2 = tab2.columns[i]
        if is_numeric_dtype(tab2[col_name2]):
            drop_col2.append(col_name2)
    for drop in drop_col2:        
        tab2.drop(columns=[drop],inplace=True)
    
    #각 테이블의 모든 것에 컬럼들에 대해 컬럼쌍 만들기
    col_comb = []
    for n1 in tab1.columns:
        for n2 in tab2.columns:
            col_comb.append([n1,n2])
    #print(col_comb)
            
        
    #각 컬럼쌍들에 대해 column_name_embedding해서 1차 거르기 True / False
    drop_CN = []
    for comb in col_comb:
        #print(comb)
        if column_name_emb(comb[0],comb[1]) < col_th:
            #print(comb)
            drop_CN.append(comb)
    #print("CN : ",col_comb)
    #print(drop_CN)
    for drop in drop_CN:
        col_comb.remove(drop)
            
    #NER해서 2차 거르기
    drop_NER = []
    for comb in col_comb:
        sent1 = make_sent(tab1,comb[0])
        sent2 = make_sent(tab2,comb[1])
        if evaluate_list(model, sent1) != evaluate_list(model, sent2): 
            drop_NER.append(comb)
    for drop in drop_NER:
        col_comb.remove(drop)
    #print("NER : ",col_comb)
    
    #containment해서 최종 거르기
    drop_cont = []
    for comb in col_comb:
        #print(comb)
        if not (containment(tab1[comb[0]].unique(),tab2[comb[1]].unique()) or containment(tab2[comb[1]].unique(),tab1[comb[0]].unique())):
            drop_cont.append(comb)
    for drop in drop_cont:
        col_comb.remove(drop)
    
    if len(col_comb) > 0 :
        return col_comb
    else:
        return False
    
    

In [15]:
import glob
table_list = []
path = 'C:\관계형테이블임베딩\데이터셋\*.*'
for file in glob.glob(path, recursive=True):
    table_list.append(file)
from itertools import combinations
tab_comb = []
for i in combinations(table_list,2):
    tab_comb.append(i)
tab_comb
    

['C:\\관계형테이블임베딩\\데이터셋\\2019_world_happiness.csv',
 'C:\\관계형테이블임베딩\\데이터셋\\Airports.csv',
 'C:\\관계형테이블임베딩\\데이터셋\\android-games.csv',
 'C:\\관계형테이블임베딩\\데이터셋\\android_games_224games.csv',
 'C:\\관계형테이블임베딩\\데이터셋\\AQI and Lat Long of Countries.csv',
 'C:\\관계형테이블임베딩\\데이터셋\\Automobile.csv',
 'C:\\관계형테이블임베딩\\데이터셋\\Best Songs on Spotify from 2000-2023.csv',
 'C:\\관계형테이블임베딩\\데이터셋\\bournemouth_venues.csv',
 'C:\\관계형테이블임베딩\\데이터셋\\CARS_1.csv',
 'C:\\관계형테이블임베딩\\데이터셋\\Car_Models.csv',
 'C:\\관계형테이블임베딩\\데이터셋\\chip_dataset.csv',
 'C:\\관계형테이블임베딩\\데이터셋\\CO2_emission.csv',
 'C:\\관계형테이블임베딩\\데이터셋\\coffee-listings-from-all-walmart-stores.csv',
 'C:\\관계형테이블임베딩\\데이터셋\\commodity_prices.csv',
 'C:\\관계형테이블임베딩\\데이터셋\\countries-table.csv',
 'C:\\관계형테이블임베딩\\데이터셋\\Covid Live.csv',
 'C:\\관계형테이블임베딩\\데이터셋\\Crop_recommendation.csv',
 'C:\\관계형테이블임베딩\\데이터셋\\Customertravel.csv',
 'C:\\관계형테이블임베딩\\데이터셋\\df_arabica_clean.csv',
 'C:\\관계형테이블임베딩\\데이터셋\\ds_salaries.csv',
 'C:\\관계형테이블임베딩\\데이터셋\\europe.csv',
 'C:\\관계형테이블임베딩\\데이터셋\\Food_

In [17]:
for tables in tab_comb:
    try:
        tab1 = pd.read_csv(tables[0],encoding='ISO-8859-1')
    except pd.errors.ParserError:
        tab1 = pd.read_csv(tables[0],encoding='ISO-8859-1',sep=';')    
    try:
        tab2 = pd.read_csv(tables[1],encoding='ISO-8859-1')
    except pd.errors.ParserError:
        tab2 = pd.read_csv(tables[1],encoding='ISO-8859-1',sep=';')
     
    
    tab1.dropna(axis=1,how='all',inplace=True)
    tab2.dropna(axis=1,how='all',inplace=True)
    if joinable_table(tab1, tab2) != False:
        print(tables[0].split("\\")[-1],tables[1].split("\\")[-1] ,joinable_table(tab1, tab2))


2019_world_happiness.csv CO2_emission.csv [['Country or region', 'Country Name']]


KeyboardInterrupt: 